In [1]:
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
print(tf.__version__)

1.15.0


### Load iris dataset

In [0]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [0]:
#convert label into "one hot" format
sess = tf_reset()
one_hot = tf.one_hot(Y, 3)
one_hot_out = sess.run(one_hot)

### Create the NN model

In [1]:
# If exist session, close it
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

In [0]:
sess = tf_reset()
def create_model():
    # create inputs
    input_ph = tf.placeholder(dtype=tf.float32, shape=[None, 4])
    output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 3])

    # create variables
    W0 = tf.get_variable(name='W0', shape=[4, 20], initializer=tf.contrib.layers.xavier_initializer())
    W1 = tf.get_variable(name='W1', shape=[20, 20], initializer=tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable(name='W2', shape=[20, 3], initializer=tf.contrib.layers.xavier_initializer())

    b0 = tf.get_variable(name='b0', shape=[20], initializer=tf.constant_initializer(0.))
    b1 = tf.get_variable(name='b1', shape=[20], initializer=tf.constant_initializer(0.))
    b2 = tf.get_variable(name='b2', shape=[3], initializer=tf.constant_initializer(0.))
    
    weights = [W0, W1, W2]
    bias = [b0, b1, b2]
    activations = [tf.nn.relu, tf.nn.relu, None]
    
    layer = input_ph
    for w, b, activation in zip(weights, bias, activations):
        layer = tf.matmul(layer, w) + b
        if activation is not None:
            layer = activation(layer)
    output_pred = layer
    
    return input_ph, output_ph, output_pred

### Define the loss & optimizer

In [0]:
input_ph, output_ph, output_pred = create_model()

# create loss
mse = tf.reduce_mean(0.5 * tf.square(output_pred - output_ph))

# create optimizer
opt = tf.train.GradientDescentOptimizer(0.01).minimize(mse)

# initialize variables
sess.run(tf.global_variables_initializer())

# create saver to save model variables
saver = tf.train.Saver()


### Train the model

In [19]:
import numpy as np
# run training
batch_size = 32
inputs = X
outputs = one_hot_out
for training_step in range(10000):
    # get a random subset of the training data
    indices = np.random.randint(low=0, high=len(inputs), size=batch_size)
    input_batch = inputs[indices]
    output_batch = outputs[indices]
    
    # run the optimizer and get the mse
    _, mse_run = sess.run([opt, mse], feed_dict={input_ph: input_batch, output_ph: output_batch})
    
    # print the mse every so often
    if training_step % 1000 == 0:
        print('{0:04d} mse: {1:.3f}'.format(training_step, mse_run))
        saver.save(sess, '/tmp/model.ckpt')

0000 mse: 3.130
1000 mse: 0.014
2000 mse: 0.012
3000 mse: 0.017
4000 mse: 0.006
5000 mse: 0.015
6000 mse: 0.010
7000 mse: 0.007
8000 mse: 0.016
9000 mse: 0.012


### The prediction result

In [0]:
# create output & accuracy
predict = tf.cast(output_pred>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, output_ph), dtype=tf.float32))

a, p, h = sess.run([accuracy, predict, output_pred], feed_dict={input_ph:inputs, output_ph:outputs})

In [21]:
print('accuracy: ', a)

accuracy:  0.98
